## to explore

po konzultaci s jayjayem:

15min has better correlations to daily returns
15min created with tbbo data
applyied rolling ma
target is moving ma, which has a memory
by retraining the model you are learning the ma and preserving the ma memory
maybe use imbalance columns
add as feature
  * anchored daily vwap
  * anchored daily ma (14)
  * just ma as features
  * forward looking MA as target (https://claude.ai/chat/69fbfe22-bc7f-4795-b964-15d10fbfca60)
  * on MA as features/targets https://chatgpt.com/c/672d03bf-ded4-8013-8349-330786edc44f
  * https://claude.ai/chat/69fbfe22-bc7f-4795-b964-15d10fbfca60 

figure out how to turn to signals


* enhance feature
  * try add number of trades, inbalance columns into features
  * try process use tbbo data for aggregation and use bbo data in ohlcv bars and build features upon it https://databento.com/docs/examples/basics-historical/midprice/example
  * datetime features?
* also keep idea of retreain the model frequently even intraday or find working interval (30+1)

This creates features and tests the model and output features to csv. Upon this csv prediction potential can be checked with other notebook and also cgb reg a class run.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')

class LibraryTradingModel:
    def __init__(self, train_days=10, test_days=1, forward_bars=5, n_classes=5):
        """
        Initialize the trading model.
        
        Parameters:
        train_days (int): Number of days for training window
        test_days (int): Number of days for testing window
        forward_bars (int): Number of bars to look ahead for returns
        n_classes (int): Number of classes to create (using percentiles)
        """
        self.train_days = train_days
        self.test_days = test_days
        self.forward_bars = forward_bars
        self.n_classes = n_classes
        self.scaler = StandardScaler()
        self.class_boundaries = None
    
    def get_date_windows(self, data):
        """
        Calculate date windows for training and testing.
        Handles any frequency of data by using datetime index.
        
        Returns:
        List of tuples: (train_start, train_end, test_start, test_end)
        """
        # Convert days to timedelta
        train_delta = timedelta(days=self.train_days)
        test_delta = timedelta(days=self.test_days)
        
        start_date = data.index[0]
        end_date = data.index[-1]
        
        windows = []
        current_start = start_date
        
        while True:
            train_end = current_start + train_delta
            test_start = train_end
            test_end = test_start + test_delta
            
            # Break if we've reached the end of data
            if test_end > end_date:
                break
                
            windows.append((current_start, train_end, test_start, test_end))
            current_start = current_start + timedelta(days=self.test_days)  # Move forward by test period
            
        return windows
    
    def prepare_features(self, df):
        """
        Create features from the price data.
        Added log returns to handle small price changes better.
        """
        features = df.copy()
        
        # Log returns for better handling of small price changes
        features['log_return'] = np.log(features['close'] / features['close'].shift(1))
        
        # Regular returns
        features['returns_1'] = features['close'].pct_change()
        features['returns_5'] = features['close'].pct_change(5)
        features['returns_20'] = features['close'].pct_change(20)
        
        # Log-based features
        features['log_return_5'] = features['log_return'].rolling(5).sum()
        features['log_return_20'] = features['log_return'].rolling(20).sum()
        
        # Moving averages
        features['sma_5'] = features['close'].rolling(5).mean()
        features['sma_20'] = features['close'].rolling(20).mean()
        
        # Volatility (using log returns for better normality)
        features['volatility_5'] = features['log_return'].rolling(5).std()
        features['volatility_20'] = features['log_return'].rolling(20).std()
        
        # Price relative to moving averages
        features['price_rel_sma5'] = features['close'] / features['sma_5']
        features['price_rel_sma20'] = features['close'] / features['sma_20']
        
        # Drop NaN values
        features = features.dropna()
        
        return features
    
    def create_target(self, df, train_data=None):
        """
        Create target variable based on forward returns.
        Enhanced to handle zero returns and ensure consistent class numbers.
        """
        # Calculate forward returns
        forward_returns = df['close'].shift(-self.forward_bars) / df['close'] - 1
        
        if train_data is None:
            # This is training data - calculate class boundaries
            returns_data = forward_returns.dropna()
            
            # Calculate initial percentile values
            percentiles = np.linspace(0, 100, self.n_classes + 1)
            initial_boundaries = np.percentile(returns_data, percentiles)
            
            # Handle zero returns specially
            zero_return_idx = np.where(initial_boundaries == 0)[0]
            if len(zero_return_idx) > 1:
                # If multiple zeros, distribute them evenly
                zero_start = zero_return_idx[0]
                zero_end = zero_return_idx[-1]
                
                # Create small spreads around zero
                epsilon = 1e-10
                spreads = np.linspace(-epsilon, epsilon, zero_end - zero_start + 1)
                initial_boundaries[zero_start:zero_end + 1] = spreads
            
            # Ensure uniqueness and proper spacing
            unique_boundaries = []
            epsilon = 1e-10
            
            for i, boundary in enumerate(initial_boundaries):
                if i == 0:  # First boundary
                    unique_boundaries.append(boundary - epsilon)
                elif i == len(initial_boundaries) - 1:  # Last boundary
                    unique_boundaries.append(boundary + epsilon)
                else:  # Middle boundaries
                    # Ensure spacing from previous boundary
                    prev_boundary = unique_boundaries[-1]
                    if abs(boundary - prev_boundary) < epsilon:
                        boundary = prev_boundary + epsilon
                    unique_boundaries.append(boundary)
            
            self.class_boundaries = np.array(unique_boundaries)
            
            # Print distribution info for debugging
            print("\nClass Boundaries Distribution:")
            class_counts = []
            for i in range(len(self.class_boundaries)-1):
                count = ((returns_data >= self.class_boundaries[i]) & 
                        (returns_data < self.class_boundaries[i+1])).sum()
                pct = count / len(returns_data) * 100
                class_counts.append(count)
                print(f"Class {i}: {self.class_boundaries[i]:.8f} to {self.class_boundaries[i+1]:.8f} "
                    f"(count: {count}, {pct:.2f}%)")
            
            # Warning if any class is empty
            min_class_count = min(class_counts)
            if min_class_count == 0:
                print("\nWARNING: Some classes have zero samples!")
                
        # Ensure we have class boundaries
        if self.class_boundaries is None:
            raise ValueError("Class boundaries not set. Must process training data first.")
        
        # Classify returns into classes
        try:
            classes = pd.cut(forward_returns, 
                            bins=self.class_boundaries, 
                            labels=range(self.n_classes),
                            include_lowest=True)
            
            # Verify we have all classes represented
            unique_classes = classes.dropna().unique()
            if len(unique_classes) < self.n_classes:
                print(f"\nWARNING: Only {len(unique_classes)} classes present in data")
                print("Classes found:", sorted(unique_classes))
                
        except Exception as e:
            print(f"Error in classification: {str(e)}")
            print("Forward returns stats:")
            print(forward_returns.describe())
            print("\nClass boundaries:")
            print(self.class_boundaries)
            raise
        
        return classes

    
    def prepare_data(self, df, train_data=None):
        """
        Prepare features and target for modeling.
        """
        # Prepare features
        features_df = self.prepare_features(df)
        
        # Create target
        target = self.create_target(features_df, train_data if train_data is not None else None)
        
        # Select feature columns
        feature_cols = ['log_return', 'returns_1', 'returns_5', 'returns_20',
                    'log_return_5', 'log_return_20',
                    'volatility_5', 'volatility_20',
                    'price_rel_sma5', 'price_rel_sma20']
        
        X = features_df[feature_cols]
        y = target
        
        # Scale features
        if train_data is None:
            X_scaled = self.scaler.fit_transform(X)
        else:
            X_scaled = self.scaler.transform(X)
        
        # Convert to DataFrame to keep column names
        X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
        
        return X_scaled, y
    
    def display_confusion_matrix(self, y_true, y_pred, title):
        """
        Display confusion matrix with detailed information.
        """
        cm = confusion_matrix(y_true, y_pred)
        cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
        
        labels = [[f'{int(cm[i, j])}\n{cm_percent[i, j]:.1f}%' 
                  if cm[i, j] > 0 else ''
                  for j in range(len(cm))]
                 for i in range(len(cm))]
        
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=labels, fmt='', cmap='Blues')
        plt.title(f'Confusion Matrix: {title}')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.show()
        
        # Print additional metrics
        print("\nPer-class metrics:")
        precision = np.diag(cm) / np.sum(cm, axis=0)
        recall = np.diag(cm) / np.sum(cm, axis=1)
        f1 = 2 * (precision * recall) / (precision + recall)
        
        metrics_df = pd.DataFrame({
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1
        }, index=[f'Class {i}' for i in range(self.n_classes)])
        
        print(metrics_df.round(3))
        print(f"\nOverall Accuracy: {np.sum(np.diag(cm)) / np.sum(cm):.3f}")

    def run_iteration(self, train_data, test_data, iteration_num):
        """
        Run a single iteration of training and testing.
        Enhanced error handling and class validation.
        """
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        
        try:
            # Reset class boundaries for each iteration
            self.class_boundaries = None
            
            print(f"\nProcessing iteration {iteration_num}")
            print(f"Training data: {train_data.index[0]} to {train_data.index[-1]}")
            print(f"Test data: {test_data.index[0]} to {test_data.index[-1]}")
            
            # Prepare training data
            X_train, y_train = self.prepare_data(train_data)
            mask_train = ~y_train.isna()
            X_train = X_train[mask_train]
            y_train = y_train[mask_train]
            
            if len(X_train) < self.forward_bars + 1:
                print(f"Warning: Iteration {iteration_num} - Insufficient training data")
                return None, None
                
            print(f"Training samples after preparation: {len(X_train)}")
            
            # Validate class distribution
            class_dist = y_train.value_counts(normalize=True).sort_index()
            print(f"Class distribution in training:")
            print(class_dist)
            
            # Check if we have all classes
            missing_classes = set(range(self.n_classes)) - set(class_dist.index)
            if missing_classes:
                print(f"\nWARNING: Missing classes in training data: {missing_classes}")
                print("Adjusting model to handle missing classes...")
                # Add one sample for each missing class to ensure model can handle all classes
                for missing_class in missing_classes:
                    X_train = pd.concat([X_train, X_train.iloc[[0]]])
                    y_train = pd.concat([y_train, pd.Series([missing_class], index=[X_train.index[-1]])])
            
            # Train the model
            model.fit(X_train, y_train)
            
            # Prepare test data
            X_test, y_test = self.prepare_data(test_data, train_data)
            
            # Make predictions
            predictions = model.predict(X_test)
            probabilities = model.predict_proba(X_test)
            
            # Validate probabilities shape
            if probabilities.shape[1] != self.n_classes:
                raise ValueError(f"Probability matrix has wrong shape: {probabilities.shape}")
            
            # Create results DataFrame
            results = pd.DataFrame({
                'predicted_class': predictions,
                'actual_class': y_test
            }, index=X_test.index)
            
            # Add probability for each class
            for i in range(self.n_classes):
                results[f'prob_class_{i}'] = probabilities[:, i]
            
            # Display iteration results
            mask = ~results['actual_class'].isna()
            if mask.any():
                print(f"\nValid test samples: {mask.sum()}")
                self.display_confusion_matrix(
                    results.loc[mask, 'actual_class'],
                    results.loc[mask, 'predicted_class'],
                    f'Iteration {iteration_num}'
                )
            
            return results, model
            
        except Exception as e:
            print(f"Error in iteration {iteration_num}: {str(e)}")
            print(f"Training data shape: {train_data.shape}")
            print(f"Test data shape: {test_data.shape}")
            return None, None

    def run_rolling_window(self, data, num_iterations=None):
        """
        Run the model using a rolling window approach.
        Handles any frequency of data using datetime index.
        """
        # Get all possible windows
        windows = self.get_date_windows(data)
        
        if num_iterations is not None:
            windows = windows[:num_iterations]
        
        all_results = {}
        all_predictions = []
        all_actuals = []
        
        for i, (train_start, train_end, test_start, test_end) in enumerate(windows):
            # Get data for this iteration using datetime indexing
            train_mask = (data.index >= train_start) & (data.index < train_end)
            test_mask = (data.index >= test_start) & (data.index < test_end)
            
            train_data = data[train_mask]
            test_data = data[test_mask]
            
            # Skip if not enough data
            min_required_bars = max(20, self.forward_bars + 1)  # minimum bars needed for features
            if len(train_data) < min_required_bars or len(test_data) < 1:
                print(f"Skipping iteration {i}: Insufficient data (train: {len(train_data)}, test: {len(test_data)} bars)")
                continue
            
            results, model = self.run_iteration(train_data, test_data, i)
            
            if results is not None:
                all_results[i] = {
                    'train_period': (train_start, train_end),
                    'test_period': (test_start, test_end),
                    'results': results,
                    'model': model
                }
                
                mask = ~results['actual_class'].isna()
                if mask.any():
                    all_predictions.extend(results.loc[mask, 'predicted_class'])
                    all_actuals.extend(results.loc[mask, 'actual_class'])
        
        # Display overall results
        if all_predictions:
            print("\n=== Overall Results Across All Iterations ===")
            self.display_confusion_matrix(
                all_actuals,
                all_predictions,
                'All Iterations Combined'
            )
        elif not all_results:
            print("No valid results generated across all iterations.")

        return all_results

    def generate_features_csv(self, data, output_filename, use_raw_target=True):
        """
        Generate raw (unscaled) features and target variables for the entire dataset and save to CSV.
        
        Parameters:
        data (pd.DataFrame): Input data with required columns (close, etc.)
        output_filename (str): Name of the output CSV file
        use_raw_target (bool): If True, use raw forward returns as target instead of classes
        
        Returns:
        pd.DataFrame: DataFrame containing all features and target variables
        """
        try:
            print("Preparing features for entire dataset...")
            
            # Prepare features
            features_df = self.prepare_features(data)
            
            # Calculate raw forward returns
            forward_returns = features_df['close'].shift(-self.forward_bars) / features_df['close'] - 1
            
            # If using classified target, create classes
            if not use_raw_target:
                self.class_boundaries = None  # Reset class boundaries
                target = self.create_target(features_df)
            else:
                target = forward_returns
            
            # Select feature columns (same as in prepare_data method)
            feature_cols = [
                'log_return', 'returns_1', 'returns_5', 'returns_20',
                'log_return_5', 'log_return_20',
                'volatility_5', 'volatility_20',
                'price_rel_sma5', 'price_rel_sma20'
            ]
            
            # Get raw features (no scaling)
            X = features_df[feature_cols]
            
            # Rename columns to match desired output format
            output_df = pd.DataFrame(X.values, columns=[f'feature_{i}' for i in range(X.shape[1])], 
                                index=X.index)
            
            # Add the target variable
            output_df['target'] = target
            
            # Add the timestamp as a column instead of index
            output_df.reset_index(inplace=True)
            output_df.rename(columns={'index': 'Open time'}, inplace=True)
            
            # Remove rows with NaN values (these would be from the initial rolling windows 
            # and forward-looking target calculation)
            output_df = output_df.dropna()
            
            # Save to CSV
            output_df.to_csv(output_filename, index=False)
            
            print(f"\nFeatures and target generated successfully!")
            print(f"Output saved to: {output_filename}")
            print(f"Total rows: {len(output_df)}")
            print(f"Features shape: {X.shape}")
            print(f"Target type: {'Raw forward returns' if use_raw_target else 'Classified'}")
            print("\nFeature columns:")
            for i, col in enumerate(feature_cols):
                print(f"feature_{i}: {col}")
            
            # Print some basic statistics about the features
            print("\nFeature statistics:")
            stats_df = X.describe()
            print(stats_df)
                
            return output_df
            
        except Exception as e:
            print(f"Error generating features: {str(e)}")
            raise

In [4]:
import vectorbtpro as vbt
import ttools as tts
#from lightweight_charts import chart, Panel, PlotDFAccessor, PlotSRAccessor
#import talib
import ttools as tts
from ttools.config import DATA_DIR
from ttools.utils import zoneNY, AggType
from ttools.loaders import load_data
from numba import jit
import pandas as pd
import numpy as np
from datetime import datetime
vbt.settings.plotting.auto_rangebreaks = True
vbt.settings.set_theme("dark")
vbt.settings.plotting["use_resampler"] = True

TTOOLS: Loaded env variables from file /Users/davidbrazda/Documents/Development/python/.env


In [9]:
#This is how to call LOAD function
symbol = ["BAC"]
#datetime in zoneNY 
day_start = datetime(2024, 10, 1, 9, 30, 0)
day_stop = datetime(2024, 10, 20, 16, 0, 0)
day_start = zoneNY.localize(day_start)
day_stop = zoneNY.localize(day_stop)

#requested AGG
resolution = 1 #12s bars
agg_type = AggType.OHLCV #other types AggType.OHLCV_VOL, AggType.OHLCV_DOL, AggType.OHLCV_RENKO
exclude_conditions = ['C','O','4','B','7','V','P','W','U','Z','F','9','M','6'] #None to defaults
minsize = 100 #min trade size to include
main_session_only = False
force_remote = False

bac_data = load_data(symbol = symbol,
                     agg_type = agg_type,
                     resolution = resolution,
                     start_date = day_start,
                     end_date = day_stop,
                     #exclude_conditions = None,
                     minsize = minsize,
                     main_session_only = main_session_only,
                     force_remote = force_remote,
                     return_vbt = False, #returns vbt object
                     verbose = True
                     )

matched agg files 1

File: BAC-AggType.OHLCV-1-2024-01-15T09-30-00-2024-10-20T16-00-00-4679BCFMOPUVWZ-100-False.parquet
Coverage: 2024-01-15 09:30:00 to 2024-10-20 16:00:00
Symbol: BAC
Agg Type: AggType.OHLCV
Resolution: 1
Excludes: 4679BCFMOPUVWZ
Minsize: 100
Main Session Only: False
--------------------------------------------------------------------------------
Loaded from agg_cache /Users/davidbrazda/Library/Application Support/v2realbot/aggcache/BAC-AggType.OHLCV-1-2024-01-15T09-30-00-2024-10-20T16-00-00-4679BCFMOPUVWZ-100-False.parquet


In [ ]:
df = bac_data["BAC"].between_time("9:30", "16:00").copy()

model = LibraryTradingModel(train_days=5, test_days=1, forward_bars=3, n_classes=3)

features_df = model.generate_features_csv(df, "features_output.csv", use_raw_target=True) #raw or classed

#results = model.run_rolling_window(df, num_iterations=3)



In [13]:
features_df

,time,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,target
0,2024-10-01 09:30:22-04:00,0.000253,0.000253,0.000253,-0.001265,2.533891e-04,-1.265983e-03,0.000263,0.000392,1.000406,1.000479,0.000507
1,2024-10-01 09:30:24-04:00,-0.000253,-0.000253,0.000380,-0.000760,3.802040e-04,-7.599747e-04,0.000212,0.000359,1.000076,1.000263,0.000760
2,2024-10-01 09:30:25-04:00,0.000405,0.000405,0.000786,0.000405,7.855958e-04,4.053917e-04,0.000250,0.000329,1.000324,1.000649,0.000228
3,2024-10-01 09:30:28-04:00,0.000355,0.000355,0.001014,0.000887,1.013428e-03,8.866933e-04,0.000263,0.000336,1.000476,1.000959,-0.000253
4,2024-10-01 09:30:29-04:00,0.000000,0.000000,0.000760,0.000760,7.599747e-04,7.599747e-04,0.000275,0.000335,1.000324,1.000921,-0.000410
...,...,...,...,...,...,...,...,...,...,...,...,...
155705,2024-10-18 15:59:52-04:00,0.000000,0.000000,0.000000,-0.000236,0.000000e+00,-2.362949e-04,0.000000,0.000132,1.000000,1.000006,0.000000
155706,2024-10-18 15:59:53-04:00,-0.000118,-0.000118,-0.000118,-0.000354,-1.181684e-04,-3.544633e-04,0.000053,0.000134,0.999905,0.999906,0.000236
155707,2024-10-18 15:59:54-04:00,0.000118,0.000118,0.000000,0.000118,4.323256e-17,1.181684e-04,0.000084,0.000111,1.000024,1.000018,0.000354
155708,2024-10-18 15:59:55-04:00,0.000000,0.000000,0.000000,0.000000,4.323256e-17,1.260385e-16,0.000084,0.000108,1.000024,1.000018,0.000168
